# AI - CA1 - Mohamad Taha Fakharian

## Goal

In [1]:
import numpy as np
import pandas as pd

In [2]:
def is_valid(x, y, n, m):
    return x >= 0 and y >= 0 and x < n and y < m

In [3]:
def specify_inspected_cells(ork_cell, ork_id, degree, grid, n, m):
    ork_x, ork_y = ork_cell
    grid[ork_x][ork_y] = 'O'
    for i in range(degree + 1):
        for j in range(degree - i + 1):
            if i == 0 and j == 0:
                continue
            for delta_i in [-1, 1]:
                for delta_j in [-1, 1]:
                    if is_valid(ork_x + delta_i * i, ork_y + delta_j * j, n , m):
                        grid[ork_x + delta_i * i][ork_y + delta_j * j] = 'X' + str(ork_id)

def prepare_grid(file_name):
    lines = []
    with open(file_name, "r") as test_file:
        for line in test_file:
            lines.append(line.rstrip("\n"))
    
    n, m = map(int, lines[0].split())
    grid = [['EMPTY' for j in range(m)] for i in range(n)]
    x_start, y_start = map(int, lines[1].split())
    grid[x_start][y_start] = 'START'
    x_end, y_end = map(int, lines[2].split())
    grid[x_end][y_end] = 'END'
    
    k, l = map(int, lines[3].split())
    degrees = []
    for i in range(k):
        x, y, c = map(int, lines[4 + i].split())
        degrees.append(c)
        specify_inspected_cells((x, y), i, c, grid, n, m)
    
    for i in range(l):
        x, y = map(int, lines[4 + k + i].split())
        grid[x][y] = 'F' + str(i)
        
    friends_end = []
    for i in range(l):
        x, y = map(int, lines[4 + k + l + i].split())
        friends_end.append((x, y))
    return grid, (x_start, y_start), friends_end, degrees

In [4]:
def reached_goal(grid, state):
    (x, y), current_damage, current_ork, current_friend, friends_status = state
    if grid[x][y] == 'END' and friends_status.find('0') == -1:
        return True
    return False

def calc_manhattan_dist(cur_cell, next_cell):
    x1, y1 = cur_cell
    x2, y2 = next_cell
    return abs(x2 - x1) + abs(y2 - y1)

def is_ok(grid, state, degrees):
    (x, y), current_damage, current_ork, current_friend, friends_status = state
    if grid[x][y] == 'O':
        return False
    if grid[x][y][0] == 'X':
        if current_damage > degrees[current_ork]:
            return False
    return True

def update_state(grid, current_state, friends_end):
    (x, y), current_damage, current_ork, current_friend, friends_status = current_state
    if current_friend != -1 and friends_end[current_friend] == (x, y):
        friends_status = friends_status[:current_friend] + '1' + friends_status[current_friend+1:]
        current_friend = -1
    
    if grid[x][y][0] == 'F' and current_friend == -1 and friends_status[int(grid[x][y][1:])] == '0':
        current_friend = int(grid[x][y][1:])
        
    if grid[x][y][0] == 'X':
        if int(grid[x][y][1:]) == current_ork:
            current_damage += 1
        else:
            current_damage = 1
            current_ork = int(grid[x][y][1:])
    else:
        current_damage = 0
        current_ork = -1
    return ((x, y), current_damage, current_ork, current_friend, friends_status)

In [5]:
import time

def get_path(end_state, start_state, tree):
    path = ''
    current_state = end_state
    while current_state != start_state:
        parent_state = tree[current_state]
        (x0, y0) = current_state[0]
        (x1, y1) = parent_state[0]
        if (x0 - x1) == 1:
            path += 'D'
        elif (x0 - x1) == -1:
            path += 'U'
        elif (y0 - y1) == 1:
            path += 'R'
        elif (y0 - y1) == -1:
            path += 'L'
        current_state = parent_state
    return path[::-1]

In [6]:
def solve_problem(method, hyperparams = None):
    tests = 4
    runs = 3
    records = pd.DataFrame(columns = ['Result length', 'Total states checked', 'Mean time(s)'])
    for test in range(tests):
        test_name = "test_0" + str(test)
        grid, (x_start, y_start), friends_end, degrees = prepare_grid(test_name + ".txt")
        total_time = 0
        start_state = ((x_start, y_start), 0, -1, -1, '0'*len(friends_end))
        for run in range(runs):
            t0 = time.time()
            found, path, total_states_checked = method(grid, start_state, friends_end, degrees, hyperparams)
            if not found:
                print("Couldn't find path!")
                break
            t1 = time.time()
            total_time += (t1 - t0)
        if found:
            print("Path for {}: {}".format(test_name, path))
            records.loc[test_name] = [len(path), total_states_checked, total_time/runs]
    return records

In [7]:
def bfs(grid, start_state, friends_end, degrees, hyperparams = None):
    q = []
    start_state = update_state(grid, start_state, friends_end)
    if reached_goal(grid, start_state):
        return True, get_path(start_state, start_state, path), total_states_checked
    total_states_checked = 0
    q.append(start_state)
    explored = {start_state}
    path = dict()
    while len(q) > 0:
        current_state = q.pop(0)
        total_states_checked += 1
        (x, y), current_damage, current_ork, current_friend, friends_status = current_state
        for delta_x in range(-1, 2):
            for delta_y in range(-1, 2):
                if abs(delta_x) != abs(delta_y) and is_valid(x + delta_x, y + delta_y, len(grid), len(grid[0])):
                    next_cell = (x + delta_x, y + delta_y)
                    next_state = (next_cell, current_damage, current_ork, current_friend, friends_status)
                    next_state = update_state(grid, next_state, friends_end)
                    if (not next_state in explored) and (is_ok(grid, next_state, degrees)):
                        path[next_state] = current_state
                        if reached_goal(grid, next_state):
                            return True, get_path(next_state, start_state, path), total_states_checked
                        explored.add(next_state)
                        q.append(next_state)
    return False, None, None

In [8]:
bfs_records = solve_problem(bfs)
bfs_records

Path for test_00: RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDURDURDURDURDRRRRRR
Path for test_01: RRRRDDLLDDDDDLDURUUUURRRRLLDDDLDDUURUURRRRDDDLDLLRRR
Path for test_02: RRRRRRRLLLLDDDDLDLLRDDUURURRRRDRDD
Path for test_03: RRDDDDDLDLDDRDURUUUUUUUURRRRRRRLLLLLLLDDDDDDDDLDURUUUURRRRRRRDDDDD


,Result length,Total states checked,Mean time(s)
test_00,48.0,6438.0,0.044997
test_01,52.0,1383.0,0.006753
test_02,34.0,380.0,0.001883
test_03,66.0,3946.0,0.028471


In [9]:
def rdls(grid, path, explored, degrees, friends_end, start_state, state, limit, total_states_checked):
    total_states_checked += 1
    if reached_goal(grid, state):
        return True, get_path(state, start_state, path), total_states_checked
    if limit == 0:
        return False, None, total_states_checked
    (x, y), current_damage, current_ork, current_friend, friends_status = state
    for delta_x in range(-1, 2):
        for delta_y in range(-1, 2):
            if abs(delta_x) != abs(delta_y) and is_valid(x + delta_x, y + delta_y, len(grid), len(grid[0])):
                next_cell = (x + delta_x, y + delta_y)
                next_state = (next_cell, current_damage, current_ork, current_friend, friends_status)
                next_state = update_state(grid, next_state, friends_end)
                if (not next_state in explored or explored[next_state] > explored[state] + 1) and (is_ok(grid, next_state, degrees)):
                    explored[next_state] = explored[state] + 1
                    path[next_state] = state
                    reached, possible_path, total_states_checked = rdls(grid, path, explored, degrees, friends_end, start_state, next_state, limit-1, total_states_checked)
                    if reached:
                        return reached, possible_path, total_states_checked
    return False, None, total_states_checked

def dls(grid, degrees, friends_end, start_state, limit, total_states_checked):
    explored = {
        start_state : 0
    }
    path = dict()
    return rdls(grid, path, explored, degrees, friends_end, start_state, start_state, limit, total_states_checked)

def ids(grid, start_state, friends_end, degrees, hyperparams = None):
    total_states_checked = 0
    for i in range(len(grid) * len(grid[0]) * len(friends_end)):
        reached, possible_path, total_states_checked = dls(grid, degrees, friends_end, start_state, i, total_states_checked)
        if reached:
            return reached, possible_path, total_states_checked
    return False, None, None

In [10]:
ids_records = solve_problem(ids)
ids_records

Path for test_00: RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDURDURDURDURDRRRRRR
Path for test_01: RRRRDDLLDDDDDLDURUUUURRRRLLDDDLDDUURUURRRRDDDLDLLRRR
Path for test_02: RRRRRRRLLLLDDDDLDLLRDDUURURRRRDRDD
Path for test_03: RRDDDDDLDLDDRDURUUUUUUUURRRRRRRLLLLLLLDDDDDDDDLDURUUUURRRRRRRDDDDD


,Result length,Total states checked,Mean time(s)
test_00,48.0,130983.0,0.546850
test_01,52.0,207801.0,1.069705
test_02,34.0,19900.0,0.101346
test_03,66.0,864222.0,4.714786
